# 🔐 Password Strength Analyzer (Entropy + Rule System)

## STEP 1 — Install Dependencies (optional)

Only needed if you want wordlists for dictionary attack.

In [ ]:
# Optional: Download wordlist (Linux only)
# !wget https://raw.githubusercontent.com/danielmiessler/SecLists/master/Passwords/Common-Credentials/10-million-password-list-top-1000000.txt -O common.txt


## STEP 2 — Password Entropy Function

In [1]:
import math
import re

def calculate_entropy(password):
    """Calculate Shannon entropy of a password."""
    if not password:
        return 0

    # Count occurrences of each character
    char_counts = {c: password.count(c) for c in set(password)}
    length = len(password)

    entropy = 0
    for count in char_counts.values():
        p = count / length
        entropy -= p * math.log2(p)

    # Convert per-character entropy to total password entropy
    return round(entropy * length, 2)


## STEP 3 — Rule Based Scoring System

In [2]:
def rule_score(password):
    score = 0
    feedback = []

    # Length rules
    if len(password) >= 8:
        score += 1
    else:
        feedback.append("❌ Use at least 8 characters.")

    if len(password) >= 12:
        score += 1
    else:
        feedback.append("⚠️ Stronger passwords have 12+ characters.")

    # Character diversity
    if re.search(r"[A-Z]", password):
        score += 1
    else:
        feedback.append("❌ Add uppercase letters (A-Z).")

    if re.search(r"[a-z]", password):
        score += 1
    else:
        feedback.append("❌ Add lowercase letters (a-z).")

    if re.search(r"[0-9]", password):
        score += 1
    else:
        feedback.append("❌ Add digits (0-9).")

    if re.search(r"[^A-Za-z0-9]", password):
        score += 1
    else:
        feedback.append("❌ Add special characters (!@#$...).")

    # Repeated patterns
    if re.search(r"(.)\1\1", password):
        feedback.append("⚠️ Avoid repeating characters (aaa, 111).")
        score -= 1
   
    # Sequences
    seq = "abcdefghijklmnopqrstuvwxyz"
    numseq = "0123456789"
    if any(password.lower() in seq[i:i+4] for i in range(len(seq)-3)):
        feedback.append("⚠️ Avoid alphabetical sequences.")
        score -= 1
    if any(password in numseq[i:i+4] for i in range(len(numseq)-3)):
        feedback.append("⚠️ Avoid numeric sequences.")
        score -= 1

    return score, feedback


## STEP 4 — Dictionary Attack Detection (Optional)

In [3]:
def dictionary_check(password, filepath="common.txt"):
    try:
        with open(filepath, "r", errors="ignore") as f:
            common = set(line.strip().lower() for line in f)
        if password.lower() in common:
            return True
    except FileNotFoundError:
        pass
    return False


## STEP 5 — Final Password Strength Analyzer

In [4]:
def analyze_password(password):
    entropy = calculate_entropy(password)
    score, feedback = rule_score(password)
    is_common = dictionary_check(password)

    # Final strength label
    if is_common:
        strength = "Very Weak (Common Password)"
        color = "🔴"
    elif entropy < 28:
        strength = "Very Weak"
        color = "🔴"
    elif entropy < 45:
        strength = "Weak"
        color = "🟠"
    elif entropy < 60:
        strength = "Moderate"
        color = "🟡"
    elif entropy < 75:
        strength = "Strong"
        color = "🟢"
    else:
        strength = "Very Strong"
        color = "🟣"

    print("\n===============================")
    print(f"Password: {password}")
    print(f"Strength: {color} {strength}")
    print(f"Entropy: {entropy} bits")
    print(f"Rule Score: {score}/6")
    print("-------------------------------")

    if feedback:
        print("Suggestions:")
        for f in feedback:
            print("•", f)
    else:
        print("✔ No improvement needed. Excellent password!")
    print("===============================\n")


## STEP 6 — Test It

In [5]:
analyze_password("Abc@123")
analyze_password("A9$dJk!pL2*")
analyze_password("password123")



Password: Abc@123
Strength: 🔴 Very Weak
Entropy: 19.65 bits
Rule Score: 4/6
-------------------------------
Suggestions:
• ❌ Use at least 8 characters.
• ⚠️ Stronger passwords have 12+ characters.


Password: A9$dJk!pL2*
Strength: 🟠 Weak
Entropy: 38.05 bits
Rule Score: 5/6
-------------------------------
Suggestions:
• ⚠️ Stronger passwords have 12+ characters.


Password: password123
Strength: 🟠 Weak
Entropy: 36.05 bits
Rule Score: 3/6
-------------------------------
Suggestions:
• ⚠️ Stronger passwords have 12+ characters.
• ❌ Add uppercase letters (A-Z).
• ❌ Add special characters (!@#$...).

